### Posterior distributions

In [19]:
# add packages and load data

library(data.table)
library(tidyverse)
library(ggridges)
library(here)
options(warn = -1)


cond <- "late"


df <- read.table(paste0('intercept',cond,'_post.txt'))
conditions <- c("early")

In [44]:
for (cond in conditions) {
    posteriors <- read.table(paste0('intercept',cond,'_post.txt'), header = TRUE)
    n.obj <- dim(posteriors)[1]
    df.long <- posteriors %>%
        gather("ROI") %>%
        group_by(ROI) %>%
        mutate(p.plus = sum(value > 0)/length(value),
               mean = mean(value),
               median = median(value)) %>%
       arrange(p.plus, median) %>%
       ungroup() %>%
       mutate(index = rep(1:length(unique(ROI)), each = nobj),
         p.plus = format(round(p.plus, 3), nsmall = 2),
         p.plus.gray = p.plus,
         p.plus.gray = replace(p.plus.gray, p.plus.gray > 0.15 & p.plus.gray < 0.85, NA),
         p.plus.gray = as.numeric(p.plus.gray)) 
}
head(df.long)

ERROR: Error: Column `index` must be length 3400000 (the number of rows) or one, not 3400085


In [46]:
length(unique(df.long$ROI))

[1] 85

In [35]:
for (cond in conditions) {
    posteriors <- read.table(paste0('intercept',cond,'_post.txt'), header = TRUE)
    n.obj <- dim(posteriors)[1]
    df.long <- posteriors %>%
          gather("ROI") %>%                                                 
          group_by(ROI) %>%
          mutate(p.plus = sum(value > 0)/length(value),
                 mean = mean(value),
                 median = median(value)) %>%                 
  arrange(p.plus, median) %>%
  ungroup() %>%
  mutate(index = rep(1:length(unique(ROI)), each = nobj),
         p.plus = format(round(p.plus, 3), nsmall = 2),
         p.plus.gray = p.plus,
         p.plus.gray = replace(p.plus.gray, p.plus.gray > 0.15 & p.plus.gray < 0.85, NA),
         p.plus.gray = as.numeric(p.plus.gray)) 
         
         
# create p+ dataframe that will mark all p+ values between 0.15 and 0.85 as NA, thus plotting gray distributions. This is done because ggplot does not allow for a range of values to be considered NA. Thus, a bit of a "cheat".

# Also formats P+ values so that all values are 3 decimals, even 0.

p.plus <- df.long %>%                                                         
  select(ROI, index, p.plus, p.plus.gray) %>%
  distinct()

# scaling for 2.3mm x 2.3mm per distribution
h <- length(unique(p.plus$ROI))*2.3
w <- length(unique(p.plus$ROI))*2.3 

ggplot(df.long, aes(x = value, 
           # y = as.numeric(index),
            group = ROI, 
            fill = p.plus.gray)) +
  geom_vline(xintercept = 0, 
             linetype="solid",
             color = "black",
             alpha = .95, 
             size = .30) +
  stat_density_ridges(quantile_lines = TRUE,
                      quantiles = 2,
                      alpha = .95,
                      scale = 2.5,
                      color = "gray",
                      size = .35,
                      rel_min_height = 0.01)
ggsave("test.pdf")

}


ERROR: Error: Column `index` must be length 3400000 (the number of rows) or one, not 3400085


In [33]:
head(df.long)

ROI,value,p.plus,mean,median,index,p.plus.gray
V65,R_IFG-2,0.000,NA,-0.463566,1,0
V65,-0.331454,0.000,NA,-0.463566,1,0
V65,-0.4567601,0.000,NA,-0.463566,1,0
V65,-0.3105275,0.000,NA,-0.463566,1,0
V65,-0.570497,0.000,NA,-0.463566,1,0
V65,-0.433908,0.000,NA,-0.463566,1,0


In [34]:
head(posteriors)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85
L__d_ant/lat_Thalamus,L__d_ant/med_Thalamus,L__v_med/post_Thalamus,L_ACC,L_ant._Caudate,L_ant._dorsal_Insula,L_ant._Hippocampus,L_ant._MCC,L_ant._Putamen,L_ant._ventral_Insula,...,R_post._Caudate,R_post._MCC,R_post._Putamen,R_pre-SMA,R_pregenual_ACC,R_v_ant/inf_Thalamus,R_v_ant/sup_Thalamus,R_v_lat/post_Thalamus,R_v_med/post_Thalamus,R_Ventral_striatum
0.126809,0.018024,-0.2297062,0.21407,0.116623,0.09877,-0.750846,0.638087,-0.0993838,-0.000923000000000007,...,-0.083759,0.244879,-0.18779176,0.300082,-0.055323,0.386591,0.294113,-0.460228,0.047161,0.157535
0.3166149,-0.0263387,0.0847579,0.1564639,0.2542919,0.2090779,-0.5812981,0.6115449,0.0583819,-0.0173072,...,-0.1302269,0.2907979,-0.0027567,0.4962209,0.0996609,0.3570129,0.4518889,-0.2617691,0.1736129,0.0982649
0.0840085,0.0799625,0.019606,0.3135075,0.2134055,0.2477015,-0.5316305,0.6332375,0.0725335,0.0462924,...,0.0125434,0.2226835,-0.3307495,0.3324235,0.0729875,0.3768105,0.4705265,-0.3105705,0.0715005,0.3023435
0.298364,-0.0763714,-0.1024872,0.068558,0.201587,0.096334,-0.698167,0.61485,-0.0216507,-0.094384,...,-0.214193,0.121724,-0.003224,0.347483,-0.0279467,0.246408,0.334625,-0.32156,0.139168,-0.0154743
0.080176,0.067346,-0.0623683,0.08758,0.214605,0.243905,-0.6712,0.599166,-0.0322022,-0.0084365,...,0.010233,0.126342,-0.356046,0.412056,0.073042,0.393577,0.265869,-0.268195,-0.0295204,0.158315
